# Import Libraries

In [1]:
from swarmintelligence import GreyWolfOptimization, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
gray_misc_dataset

{'AirplaneF16.tiff': array([[ 65, 199, 193, ..., 166, 164, 148],
        [ 54, 196, 192, ..., 179, 155, 140],
        [ 54, 197, 194, ..., 190, 159, 141],
        ...,
        [ 67, 213, 213, ..., 184, 176, 151],
        [ 68, 214, 214, ..., 166, 184, 167],
        [ 89,  86,  88, ..., 112, 111, 107]], dtype=uint8),
 'Lena.png': array([[162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        [162, 162, 162, ..., 170, 155, 128],
        ...,
        [ 43,  43,  50, ..., 104, 100,  98],
        [ 44,  44,  55, ..., 104, 105, 108],
        [ 44,  44,  55, ..., 104, 105, 108]], dtype=uint8),
 'Male.tiff': array([[130, 138, 140, ..., 115, 113, 111],
        [139, 134, 131, ..., 110, 117, 116],
        [141, 127, 131, ..., 109, 108, 107],
        ...,
        [102, 103, 103, ...,  76,  80,  89],
        [106, 102, 109, ...,  72,  69,  82],
        [ 98, 105, 116, ...,  85,  81,  89]], dtype=uint8),
 'Mandrill.tiff': array([[145,  56,  49, ..., 113, 153, 177],

# Load Optimal Parameter

In [4]:
optimal_params = {
    'maxIteration': 100,
    'wolfSize': 25
}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['mMasi_entropy']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds':[],
    'Mean CPU_time (seconds)': [],
    'Mean Fitness':[],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions': [],
    'CPU_time (seconds)': [],
    'Fitness':[],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in gray_misc_dataset.items():
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                fitness_values = []
                list_best_thresholds = []
                list_fitness = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo = GreyWolfOptimization(
                        k=threshold,
                        wolfSize=optimal_params['wolfSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs, best_thresholds = gwo.fit_run(image_array)
                    et = time.time()
                    regions = Utilization().digitize(image_array, best_thresholds)
                    list_cpu_time.append(et-st)
                    list_best_thresholds.append(best_thresholds)
                    fitness_values.append(gwo.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, regions))
                    list_rmse.append(rmse(image_array, regions))
                    list_psnr.append(psnr(image_array, regions))
                    list_ssim.append(ssim(image_array, regions)[0])
                    list_uqi.append(uqi(image_array, regions))
                    print(gwo.get_params_training_()['best_fitness_tracking'][-1])
                
                mean_best_thresholds = np.round(np.array(list_best_thresholds).mean(axis=0)).astype('int')
                mean_regions=Utilization().digitize(image_array, mean_best_thresholds)
                gwo_evaluation_results['Mean best_thresholds'].append(mean_best_thresholds)
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean Fitness'].append(np.mean(np.array(fitness_values)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions'].append(mean_regions)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['Fitness'].append(fitness_values)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean(np.array(fitness_values))}")

	 AirplaneF16.tiff | Threshold =  2
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
1948.7183902321062
Mean Fitness = 1948.7183902321058
	 AirplaneF16.tiff | Threshold =  3
2024.8375098105803
2024.8375098105803
2024.805475833105
2024.8020017977378
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.805475833105
2024.805475833105
2024.8020017977378
2024.805475833105
2024.8375098105803
2024.7871768318307
2024.805475833105
2024.8375098105803
2024.8375098105803
2024.8375098105803
2024.8

3254.63925514276
3208.6590390722986
3254.478618629125
Mean Fitness = 3253.0170751034043
	 Mandrill.tiff | Threshold =  2
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
1549.0271327708897
Mean Fitness = 1549.02713277089
	 Mandrill.tiff | Threshold =  3
1639.466171099567
1639.463360998692
1639.466171099567
1639.466171099567
1639.463360998692
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.466171099567
1639.46336

4217.914800387409
4217.937222685617
4217.914800387409
4218.026068576767
4217.961054809453
4218.037279885386
4218.023954697983
4218.026068576767
4218.037279885386
Mean Fitness = 4217.990413607707


## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds,Mean CPU_time (seconds),Mean Fitness,Mean MSE,Mean RMSE,Mean PSNR,Mean SSIM,Mean UQI,Regions,CPU_time (seconds),Fitness,MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,mMasi_entropy,max,"[113, 173]",1.994300,1948.718390,2270.640518,47.651238,14.569320,0.794411,0.940220,"[[113, 255, 255, 255, 255, 255, 255, 255, 255,...","[1.4773640632629395, 1.985032081604004, 1.6016...","[1948.7183902321062, 1948.7183902321062, 1948....","[2270.6405181884766, 2270.6405181884766, 2270....","[47.651238369936166, 47.651238369936166, 47.65...","[14.569319775617668, 14.569319775617668, 14.56...","[0.7944112258870609, 0.7944112258870609, 0.794...","[0.9402201034707982, 0.9402201034707982, 0.940..."
1,AirplaneF16.tiff,3,mMasi_entropy,max,"[93, 145, 191]",2.294452,2024.824312,1769.143274,42.060707,15.653371,0.778350,0.953324,"[[93, 255, 255, 191, 191, 191, 255, 191, 255, ...","[1.4229137897491455, 2.050802707672119, 1.8154...","[2024.8375098105803, 2024.8375098105803, 2024....","[1774.3395538330078, 1774.3395538330078, 1741....","[42.12291008267363, 42.12291008267363, 41.7354...","[15.640436268576428, 15.640436268576428, 15.72...","[0.77934507049225, 0.77934507049225, 0.7739586...","[0.9532349424759257, 0.9532349424759257, 0.953..."
2,AirplaneF16.tiff,4,mMasi_entropy,max,"[84, 129, 172, 203]",1.999571,2070.009625,1094.274517,33.079074,17.739933,0.732447,0.969394,"[[84, 203, 203, 203, 203, 203, 203, 203, 203, ...","[1.7784755229949951, 1.9516115188598633, 1.974...","[2070.0794736113085, 2070.0794736113085, 2070....","[1094.045597076416, 1094.045597076416, 1095.57...","[33.07636009412789, 33.07636009412789, 33.0994...","[17.74044938188586, 17.74044938188586, 17.7343...","[0.7326687134455045, 0.7326687134455045, 0.732...","[0.9693723689797689, 0.9693723689797689, 0.969..."
3,AirplaneF16.tiff,5,mMasi_entropy,max,"[69, 107, 143, 180, 205]",1.981385,2095.876007,910.496710,30.173393,18.538616,0.745191,0.975373,"[[69, 205, 205, 205, 180, 205, 205, 205, 205, ...","[1.9899523258209229, 1.9685134887695312, 2.049...","[2096.094632054594, 2095.823513427768, 2096.14...","[904.9572067260742, 903.3267440795898, 907.287...","[30.082506656295685, 30.05539459197949, 30.121...","[18.56452317930264, 18.572354926323307, 18.553...","[0.7455589174436703, 0.745488508210734, 0.7452...","[0.9755016242588636, 0.9754258032251908, 0.975..."
4,Lena.png,2,mMasi_entropy,max,"[93, 151]",1.864723,1961.822013,2813.567070,53.043068,13.638231,0.612182,0.883309,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...","[2.069375514984131, 1.737011194229126, 1.83244...","[1961.8220125153634, 1961.8220125153634, 1961....","[2813.567070007324, 2813.567070007324, 2813.56...","[53.04306806744237, 53.04306806744237, 53.0430...","[13.638230885124642, 13.638230885124642, 13.63...","[0.6121817460412936, 0.6121817460412936, 0.612...","[0.8833090574580458, 0.8833090574580458, 0.883..."
5,Lena.png,3,mMasi_entropy,max,"[81, 127, 171]",1.962188,2128.299351,1250.170480,35.357745,17.161114,0.706230,0.933618,"[[171, 171, 171, 171, 171, 171, 171, 171, 171,...","[1.9514899253845215, 1.8585946559906006, 1.636...","[2128.3048672466566, 2128.284181016679, 2128.2...","[1249.3418655395508, 1252.4491691589355, 1252....","[35.346030407098766, 35.389958592218434, 35.38...","[17.16399067408044, 17.153202518789254, 17.153...","[0.7058656377663244, 0.7072313681226016, 0.707...","[0.9334425202424386, 0.9340987864621145, 0.934..."
6,Lena.png,4,mMasi_entropy,max,"[75, 114, 145, 180]",2.207486,2191.864388,791.834502,28.139387,19.144563,0.753916,0.954029,"[[180, 180, 180, 180, 180, 180, 180, 180, 180,...","[2.460064172744751, 2.4985342025756836, 2.2568...","[2191.874720070565, 2191.874720070565, 2191.87...","[795.6595458984375, 795.6595458984375, 795.659...","[28.2074377762043, 28.2074377762043, 28.207437...","[19.12353083289055, 19.12353083289055, 19.1235...","[0.7534048142269026, 0.7534048142269026, 0.753...","[0.9539824804728553, 0.9539824804728553, 0.953..."
7,Lena

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/GWO/GWO_masi_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/GWO/GWO_masi_dict.pkl', 'wb'))